In [1]:
# CIEMSS Demo Query:

In [ ]:
# struct DemoSIRQuery <: Query
#     mandate_multiplier::Number
#     threshold::Number
# end

# function interpret(query::DemoSIRQuery, model::T) where {T<:ODEProbabilisticModel}
#     twin_model = twinify(model)

#     mandate_multiplier = query.mandate_multiplier
#     threshold = query.threshold 

#     new_expr = :((infected_cf/N > $threshold) * (effective_contact_rate * $mandate_multiplier)
#                 + (infected_cf/N <= $threshold) * effective_contact_rate)

#     intervention = ODEStaticIntervention(:effective_contact_rate_cf, 0, new_expr)
#     intervened_model = intervene(twin_model, intervention)

#     queried_model = add_variable(intervened_model, :(effect_of_adaptive_mask_policy), 1, :(100 * ((infected_cf - infected) - effect_of_adaptive_mask_policy)))

#     return queried_model
# end

In [ ]:
# AlgebraicJulia Workflow Wiring Diagram Example:

In [ ]:
# begin
#     # Form Workflow presentation of FreeBiproductCategory
#     @present Workflow(FreeBiproductCategory) begin
#         (File,LRN,TSpan,ODEProb,ODESol)::Ob 
#         MTKLoadLRN::Hom(File,LRN)
#         MTKFormODEProb::Hom(LRN⊗TSpan,ODEProb)
#         MTKSolveODE::Hom(ODEProb,ODESol)
#     end

#     # Form wiring diagram of load_form_sim Workflow
#     load_form_sim = @program Workflow (f::File,ts::TSpan) begin # 
#         lrn = MTKLoadLRN(f)
#         ode_prob = MTKFormODEProb(lrn,ts)
#         ode_sol = MTKSolveODE(ode_prob)
#         return ode_sol 
#     end

#     # Serialize program wiring diagram
#     # write_json_graph(load_form_sim,"diagram_load_form_sim.json") 
#     write_json_acset(load_form_sim.diagram, "diagram_load_form_sim.json")
# end

In [2]:
using Catlab, Catlab.Theories
using Catlab.CategoricalAlgebra
using Catlab.Graphics
using Catlab.Graphics: Graphviz
import Catlab.CategoricalAlgebra: migrate!
using Catlab.WiringDiagrams
using Catlab.Programs
using Catlab.Programs.RelationalPrograms

In [ ]:
# Form Workflow presentation of FreeBiproductCategory
@present Workflow(FreeBiproductCategory) begin
    
    (Output_Var, Intervention_Variable, Intervention_Derivative_Order, Intervention_Assignment, Query, Model, Twin_Model, Intervention, Twin_Model_Int)::Ob
    
    ConstructQuery::Hom(Mandate_Multiplier⊗Threshold, Query)
    
    Twinify::Hom(Model, Twin_Model)
    
    ODEStaticIntervention::Hom(Intervention_Variable⊗Query, Intervention)
    
    Intervene::Hom(Twin_Model⊗Intervention, Twin_Model_Int)
    
end

In [ ]:
# Form wiring diagram of Workflow
demo_workflow = @program Workflow (output_var::Output_Var, intervention_variable::Intervention_Variable, intervention_derivative_order::Intervention_Derivative_Order, intervention_assignment::Intervention_Assignment, model::Model) begin
    
    query = ConstructQuery(output_var, intervention_variable, intervention_derivative_order, intervention_assignment)
    
    twin_model = Twinify(model)
    
    intervention = ODEStaticIntervention(intervention_variable, query)
    
    twin_model_int = Intervene(twin_model, intervention)
    
    return twin_model_int
end

In [ ]:
# Serialize program wiring diagram
write_json_acset(demo_workflow.diagram, "demo_dwd.json")